In [ ]:
from util import *

In [ ]:
big_run=False

## Read the ntuple

In [ ]:
if big_run:
    d = uproot.open('/home/sioni/Work/corrl2-jet-multi/src/GlobalCorrelator/emp_examples/Layer2/testbench/jet/data/debugPF_TTbar_PU200.110X_v2.root')['Events']
else:
    d = uproot.open('/home/sioni/Work/corrl2-jet-multi/src/GlobalCorrelator/emp_examples/Layer2/testbench/jet/data/debugPF_TTbar_PU200.110X_v2_job1.root')['Events']

pups, jets = pups_and_jets(d)
pups_e, jets_e = pups_and_jets(d, True)

regions = DemoRegions()

## Write an emp-fwk pattern file & run sim

In [ ]:
if big_run:
    N = int(np.ceil(len(pups)/1000))
    for I in range(N):
        print(I)
        l = I*1000
        h = (I+1)*1000 if I < N-1 else -1
        write_and_run(pups[l:h], regions, 300)
        copyfile('/home/sioni/Work/corrl2-jet-multi/proj/jet-test/Jets.txt', '/home/sioni/Work/corrl2-jet-multi/src/GlobalCorrelator/emp_examples/Layer2/testbench/jet/data/Jets-{}.txt'.format(I))
        copyfile('/home/sioni/Work/corrl2-jet-multi/proj/jet-test/Regionizer-EventParticles.txt', '/home/sioni/Work/corrl2-jet-multi/src/GlobalCorrelator/emp_examples/Layer2/testbench/jet/data/Regionizer-EventParticles-{}.txt'.format(I))
else:
    write_and_run(pups, regions, 100)

In [ ]:
write_pattern_file(pups[:1], regions, mux=True, link_map=None, apx=True) #regions.link_map())

## Write an HLS simulation input file

In [ ]:
f = open('source_hr.txt', 'w')
for i in range(0,3):
    write_event_hr(f, pups[i], jets_e[i])
#write_event_hr(f, pups_msim[0], jets[0])
f.close()

## Debug space

### Check the writer

In [ ]:
pups_muxed = list(regions.pack_event_mux(pups[16], 0, True))
pups_muxed_v = [pup.toVHex(True) for frame in pups_muxed for pup in frame]
pups_muxed_flat = [pup for frame in pups_muxed for pup in frame if pup.pt > 0]
print(len(pups_muxed_flat))
print(len(pups[16]))

In [ ]:
pups_muxed[14]

In [ ]:
for i, frame in enumerate(pups_muxed):
    for j, pup in enumerate(frame):
        if pup.pt == 3.75:
            print(pup.toVHex(True), i, j)

### Check the deregionizer

In [ ]:
def get_clocks(regionparticles):
    # Get the clock cycles on which to query deregionizer output
    clocks = []
    prevclock = regionparticles.Clock.unique()[0]
    for clock in regionparticles.Clock.unique():
        if clock - prevclock > 1:
            clocks.append(prevclock)
        prevclock = clock
    return clocks

def get_pups_at_clock(regionparticles, clock):
    rp = regionparticles[regionparticles.Clock == clock]
    return [PuppiCand(pt/4,eta/100,phi/100) for (pt,eta,phi) in zip(rp.pt, rp.eta, rp.phi)]

def get_deregionizer(regionparticles, clock=None):
    if clock is not None or len(regionparticles.Clock.unique()) == 1:
        return get_pups_at_clock(regionparticles, clock)
    else:
        pups = []
        for clock in get_clocks(regionparticles):
            pups.append(get_pups_at_clock(regionparticles, clock))
        return pups

regionizer_in = pandas.read_csv('../../../../../../proj/jet-test/Regionizer-Inputs.txt', skiprows=1, delim_whitespace=True)
pups_in = [PuppiCand(pt/4,eta/100,phi/100) for (pt,eta,phi) in zip(regionizer_in.pt, regionizer_in.eta, regionizer_in.phi)]

regionparticles = pandas.read_csv('../../../../../../proj/jet-test/Regionizer-EventParticles.txt', skiprows=1, delim_whitespace=True)

pups_msim = get_deregionizer(regionparticles)
pups_r = [regions.fiducial(pups_ev) for pups_ev in pups]
pups_in = [[p for re in regions.regionize(ev, truncate=16) for rp in re for p in rp] for ev in pups]

In [ ]:
print(np.all([np.all([pup in pups_msim[i] for pup in pups_in[i]]) for i in range(len(pups_msim))]))
missing = np.array([(i, np.argwhere([not pup in pups_msim[i] for pup in pups_in[i]])) for i in range(len(pups_msim))])

In [ ]:
i = 0
print(len(pups_msim[i]), len(pups[i]))

In [ ]:
pups_msim[0]

In [ ]:
ev = regions.regionize(pups[i])
for i in range(len(ev)):
    for j in range(len(ev[i])):
        for k in range(len(ev[i][j])):
            if ev[i][j][k].pt == 42.0:
                print(i, j, ev[i][j][k])
                print(i, j, ev[i][j][k].toVHex(True))

In [ ]:
a = [p for p in sorted(pups_msim[i], reverse=True, key=lambda x : x.pt)]
b = [p for p in sorted(pups[i], reverse=True, key=lambda x : x.pt)]
b = [PuppiCand(p.pt, np.floor(p.eta*100)/100, np.floor(p.phi*100)/100) for p in b]
for j in range(len(pups[i])):
    print(a[j], b[j])

### Check the jets

In [ ]:
if big_run:
    N = int(np.ceil(len(pups)/1000))
    for i in range(N):
        d = pandas.read_csv('data/Jets-{}.txt'.format(i), skiprows=1, delim_whitespace=True)
        if i == 0:
            jets_p = d
        else:
            jets_p = jets_p.append(d, ignore_index=True)
else:
    jets_p = pandas.read_csv('../../../../../../proj/jet-test/Jets.txt', skiprows=1, delim_whitespace=True)

In [ ]:
jets_msim = []
if big_run:
    # Since each 1000 events is a different file, there is some duplication of the clock cycle on which the jets appear
    # Slice 10 jets each time (i in the outer loop means extract all jets from 1 file, then go to next file)
    N = int(np.ceil(len(pups)/1000))
    for i in range(N): 
        for c in jets_p.Clock.unique():
            j = jets_p[jets_p.Clock == c]
            jets_msim.append([Jet(pt/4,eta/100,phi/100) for (pt,eta,phi) in list(zip(j.pt, j.eta, j.phi))[10*i:10*(i+1)] if pt>0])    
    jets_msim = jets_msim[0:len(jets_e)]
else:
    for c in jets_p.Clock.unique():
        j = jets_p[jets_p.Clock == c]
        jets_msim.append([Jet(pt/4,eta/100,phi/100) for (pt,eta,phi) in zip(j.pt, j.eta, j.phi) if pt>0])

## Load the hardware data!

In [ ]:
pt = np.load('hw-data/pt.npy', allow_pickle=True)
eta = np.load('hw-data/eta.npy', allow_pickle=True)
phi = np.load('hw-data/phi.npy', allow_pickle=True)
jets_hw = Jet.fromUproot(pt, eta, phi, 5, doround=False)
jets_e = jets_e[:len(jets_hw)]
jets = jets[:len(jets_hw)]
jets_msim = jets_hw

In [ ]:
len(jets_hw)

In [ ]:
i = 0
print("MSIM")
for j in jets_msim[i]:
    print(j)
print("Emulator")
for j in jets_e[i]:
    print(j)
print("CMSSW")
for j in jets[i]:
    print(j)

## Some plots

In [ ]:
sim_pt = [j.pt for ev in jets_msim for j in ev]
sim_eta = [j.eta for ev in jets_msim for j in ev]
sim_phi = [j.phi for ev in jets_msim for j in ev]
emu_pt = [j.pt for ev in jets_e[:len(jets_msim)] for j in ev]
emu_eta = [j.eta for ev in jets_e[:len(jets_msim)] for j in ev]
emu_phi = [j.phi for ev in jets_e[:len(jets_msim)] for j in ev]
flo_pt = [j.pt for ev in jets[:len(jets_msim)] for j in ev]
flo_eta = [j.eta for ev in jets[:len(jets_msim)] for j in ev]
flo_phi = [j.phi for ev in jets[:len(jets_msim)] for j in ev]

In [ ]:
pt_lim = max(max(sim_pt), max(emu_pt))
bins = np.linspace(0, pt_lim, 75)
w = bins[1] - bins[0]
h_sim, b = np.histogram(sim_pt, bins=bins)
h_emu, b = np.histogram(emu_pt, bins=bins)
h_flo, b = np.histogram(flo_pt, bins=bins)
plt.bar(bins[:-1]+w/2, h_emu, width=w, label='CMSSW Emulator', alpha=0.5)
plt.bar(bins[:-1]+w/2, h_flo, width=w, label='CMSSW float', alpha=0.5)
plt.bar(bins[:-1]+w/2, h_sim, width=w, label='FW Modelsim', alpha=0.5)
plt.semilogy()
plt.xlabel('pT / GeV')
plt.ylabel('N jets')
plt.xlim((0, pt_lim))
plt.legend()
plt.savefig('pt.pdf')

In [ ]:
bins = np.linspace(-3.14, 3.14, 50)
w = bins[1] - bins[0]
h_sim, b = np.histogram(sim_phi, bins=bins)
h_emu, b = np.histogram(emu_phi, bins=bins)
h_flo, b = np.histogram(flo_phi, bins=bins)
plt.bar(bins[:-1]+w/2, h_flo, width=w, label='CMSSW float', alpha=0.5)
plt.bar(bins[:-1]+w/2, h_emu, width=w, label='CMSSW emulator', alpha=0.5)
plt.bar(bins[:-1]+w/2, h_sim, width=w, label='FW Modelsim', alpha=0.5)
plt.semilogy()
plt.xlabel('phi')
plt.ylabel('N jets')
plt.xlim((-3.14, 3.14))
plt.ylim((1, max(h_sim)*1.5))
plt.legend()
plt.savefig('phi.pdf')

In [ ]:
bins = np.linspace(-5, 5, 50)
w = bins[1] - bins[0]
h_sim, b = np.histogram(sim_eta, bins=bins)
h_emu, b = np.histogram(emu_eta, bins=bins)
h_flo, b = np.histogram(flo_eta, bins=bins)
plt.bar(bins[:-1]+w/2, h_flo, width=w, label='CMSSW float', alpha=0.5)
plt.bar(bins[:-1]+w/2, h_emu, width=w, label='CMSSW emulator', alpha=0.5)
plt.bar(bins[:-1]+w/2, h_sim, width=w, label='FW Modelsim', alpha=0.5)
plt.semilogy()
plt.xlabel('eta')
plt.ylabel('N jets')
plt.xlim((-3.5, 3.5))
plt.legend()
plt.savefig('eta.pdf')

## Evaluate the difference between jets

In [ ]:
djets = []
matched = []
perfect_matched = []
fakes = []
drmin = 0.1

for i, jets_msimi in enumerate(jets_msim):
    jetsi = jets_e[i]
    matchedi = np.zeros(len(jetsi), dtype=bool)
    pmatchedi = np.zeros(len(jetsi), dtype=bool)
    for jet in jets_msimi:
        djetsi = []
        dphi = np.array([jet.phi - j.phi for j in jetsi])
        dphi = (dphi + np.pi) % (2 * np.pi) - np.pi
        deta = np.array([jet.eta - j.eta for j in jetsi])
        dr = np.sqrt(dphi**2 + deta**2)
        iclosest = np.argmin(dr)
        if dr[iclosest] <= drmin:
            jet_closest = jetsi[iclosest]
            dpt = jet.pt - jet_closest.pt
            if np.abs(dpt) / jet.pt <= 0.2:
                jet.dpt = dpt
                jet.deta = deta[iclosest]
                jet.dphi = dphi[iclosest]
                djets.append(jet)
                matchedi[iclosest] = True
        if np.isin(jet,jetsi):
            pmatchedi[iclosest] = True
        else:
            fakes.append(jet)
    matched.append(matchedi)
    perfect_matched.append(pmatchedi)
djets = np.array(djets)

In [ ]:
dpt = np.array([j.dpt for j in djets])
pt = np.array([j.pt for j in djets])
dpt_lim = max(np.abs(dpt))
pt_lim = max(np.abs(pt))

bins_dpt = np.linspace(-dpt_lim, dpt_lim, 50)
bins_pt = np.linspace(0, pt_lim, 50)
#bins=np.meshgrid(bins_pt, bins_dpt)
#w = bins[1] - bins[0]
h, bpt, bdpt = np.histogram2d(dpt, pt, bins=[bins_dpt, bins_pt])
h = np.flip(h, 0)
ax = plt.imshow(h, extent=[bins_pt[0], bins_pt[-1], bins_dpt[0], bins_dpt[-1]],
           norm=colors.LogNorm(vmin=1, vmax=h.max()),
           aspect='auto')
plt.colorbar()
plt.xlabel('pT / GeV')
plt.ylabel('FW - CMSSW pT / GeV')
plt.savefig('dpt.pdf')

In [ ]:
deta = np.array([j.deta for j in djets])
eta_lim = max(np.abs(deta))
bins = np.linspace(-eta_lim, eta_lim, 50)
w = bins[1] - bins[0]
h, b = np.histogram(deta, bins=bins)
plt.bar(bins[:-1]+w/2, h, width=w, label='delta eta (FW-CMSSW)', alpha=0.5)
plt.semilogy()
plt.xlabel('eta')
plt.ylabel('N jets')
plt.xlim((-eta_lim, eta_lim))
plt.legend()
plt.savefig('deta.pdf')

In [ ]:
dphi = np.array([j.dphi for j in djets])
phi_lim = max(np.abs(dphi))
bins = np.linspace(-phi_lim, phi_lim, 50)
w = bins[1] - bins[0]
h, b = np.histogram(dphi, bins=bins)
plt.bar(bins[:-1]+w/2, h, width=w, label='delta phi (FW-CMSSW)', alpha=0.5)
plt.semilogy()
plt.xlabel('phi')
plt.ylabel('N jets')
plt.xlim((-phi_lim, phi_lim))
plt.legend()
plt.savefig('dphi.pdf')

## Unmatched jets

In [ ]:
#sim_jets = np.array([j for ji in jets[:20000] for j in ji])
#jet_event_idx = np.array([i for i in range(20000) for j in jets[i]])
matched = np.array([m for mi in matched for m in mi ])
perfect_matched = np.array([m for mi in perfect_matched for m in mi ])
sim_jets = np.array([j for ji in jets_e for j in ji])
jet_event_idx = np.array([i for i in range(len(jets_e)) for j in jets_e[i]])
jets_unmatched = sim_jets[~perfect_matched]
jet_unm_event_idx = jet_event_idx[~matched]
jet_unpm_event_idx = jet_event_idx[~perfect_matched]
pt = [j.pt for j in jets_unmatched]
eta = [j.eta for j in jets_unmatched]
phi = [j.phi for j in jets_unmatched]

In [ ]:
print(float(sum(matched)) / len(matched))
print(float(sum(perfect_matched)) / len(perfect_matched))

In [ ]:
pt_lim = max(pt)
bins = np.linspace(0, pt_lim, 75)
w = bins[1] - bins[0]
h, b = np.histogram(pt, bins=bins)
plt.bar(bins[:-1]+w/2, h, width=w, label='CMSSW float', alpha=0.5)
plt.semilogy()
plt.xlabel('pT / GeV')
plt.ylabel('N jets')
plt.xlim((0, pt_lim))
plt.legend()
plt.savefig('pt_um.pdf')

In [ ]:
bins = np.linspace(-5, 5, 50)
w = bins[1] - bins[0]
h, b = np.histogram(eta, bins=bins)
plt.bar(bins[:-1]+w/2, h, width=w, label='CMSSW float', alpha=0.5)
plt.semilogy()
plt.xlabel('eta')
plt.ylabel('N jets')
plt.xlim((-3.5, 3.5))
plt.legend()
plt.savefig('eta_um.pdf')

In [ ]:
bins = np.linspace(-3.14, 3.14, 50)
w = bins[1] - bins[0]
h, b = np.histogram(phi, bins=bins)
plt.bar(bins[:-1]+w/2, h, width=w, label='CMSSW float', alpha=0.5)
plt.semilogy()
plt.xlabel('phi')
plt.ylabel('N jets')
plt.xlim((-3.14, 3.14))
plt.legend()
plt.savefig('phi_um.pdf')

## More debugging

In [ ]:
i = 2
j = jet_unpm_event_idx[i]
print(j)
print(len(pups[j]))

In [ ]:
jets_msim[j]

In [ ]:
jets_e[j]

In [ ]:
[jet.pt*4 for jet in jets_e[j]]

In [ ]:
len(pups) / 25